In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
!wget 'https://d8it4huxumps7.cloudfront.net/uploads/files/amex/Amex_Campus_Challenge_Train.csv'

--2023-09-16 12:33:43--  https://d8it4huxumps7.cloudfront.net/uploads/files/amex/Amex_Campus_Challenge_Train.csv
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 18.67.76.46, 18.67.76.93, 18.67.76.108, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|18.67.76.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4356138899 (4.1G) [text/csv]
Saving to: ‘Amex_Campus_Challenge_Train.csv’

Amex_Campus_Challen 100%[===================>]   4.06G  20.0MB/s    in 3m 49s  

2023-09-16 12:37:32 (18.2 MB/s) - ‘Amex_Campus_Challenge_Train.csv’ saved [4356138899/4356138899]



In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=096b091353dcf4dbaef21642efe326b7a5b5d83ea27a4e6ad91ce27c5ce3ac75
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark

In [ ]:
df = spark.read.csv('Amex_Campus_Challenge_Train.csv', header=True, inferSchema=True)


count = df.count()

count


12229978

In [ ]:
# Print the schema of the DataFrame
df.printSchema()

root
 |-- ind_recommended: integer (nullable = true)
 |-- activation: integer (nullable = true)
 |-- customer_digital_activity_04: double (nullable = true)
 |-- customer_spend_01: double (nullable = true)
 |-- customer_industry_spend_01: double (nullable = true)
 |-- customer_industry_spend_02: double (nullable = true)
 |-- customer_industry_spend_03: double (nullable = true)
 |-- customer_industry_spend_04: double (nullable = true)
 |-- customer_industry_spend_05: double (nullable = true)
 |-- customer_spend_02: double (nullable = true)
 |-- customer_spend_03: double (nullable = true)
 |-- customer_merchant_02: double (nullable = true)
 |-- customer_merchant_01: double (nullable = true)
 |-- customer_spend_04: double (nullable = true)
 |-- customer_spend_05: double (nullable = true)
 |-- customer_spend_06: double (nullable = true)
 |-- customer_spend_07: double (nullable = true)
 |-- merchant_spend_01: double (nullable = true)
 |-- merchant_spend_02: double (nullable = true)
 |-- merc

In [ ]:
# Drop the 'customer' and 'merchant' columns
columns_to_drop = ['customer', 'merchant']
df = df.drop(*columns_to_drop)
df.printSchema()

root
 |-- ind_recommended: integer (nullable = true)
 |-- activation: integer (nullable = true)
 |-- customer_digital_activity_04: double (nullable = true)
 |-- customer_spend_01: double (nullable = true)
 |-- customer_industry_spend_01: double (nullable = true)
 |-- customer_industry_spend_02: double (nullable = true)
 |-- customer_industry_spend_03: double (nullable = true)
 |-- customer_industry_spend_04: double (nullable = true)
 |-- customer_industry_spend_05: double (nullable = true)
 |-- customer_spend_02: double (nullable = true)
 |-- customer_spend_03: double (nullable = true)
 |-- customer_merchant_02: double (nullable = true)
 |-- customer_merchant_01: double (nullable = true)
 |-- customer_spend_04: double (nullable = true)
 |-- customer_spend_05: double (nullable = true)
 |-- customer_spend_06: double (nullable = true)
 |-- customer_spend_07: double (nullable = true)
 |-- merchant_spend_01: double (nullable = true)
 |-- merchant_spend_02: double (nullable = true)
 |-- merc

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql.functions import rand

target_count = df.filter(col("activation") == 1).count()

# Sample a random subset of rows with activation value 0
sampled_rows = df.filter(col("activation") == 0).orderBy(rand()).limit(target_count)

# Union the sampled subset with rows with activation value 1
balanced_df = df.filter(col("activation") == 1).union(sampled_rows)


In [ ]:
import pandas as pd
df = balanced_df.toPandas()

# Display the first few rows of the Pandas DataFrame
print(df.head())

   ind_recommended  activation  customer_digital_activity_04  \
0                0           1                           NaN   
1                1           1                           3.0   
2                0           1                          12.0   
3                0           1                           NaN   
4                0           1                           2.0   

   customer_spend_01  customer_industry_spend_01  customer_industry_spend_02  \
0          38.177478                   27.790924                        84.0   
1         162.456962                  176.506000                         4.0   
2          59.711176                   21.246636                        66.0   
3          76.956000                   20.618261                        20.0   
4          49.924286                   79.792000                         2.0   

   customer_industry_spend_03  customer_industry_spend_04  \
0                     6614.24                       238.0   
1           

In [ ]:
df = pd.DataFrame(df)
df

,ind_recommended,activation,customer_digital_activity_04,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,...,distance_04,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_digital_activity_01,merchant_spend_10,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05
0,0,1,NaN,38.177478,27.790924,84.0,6614.24,238.0,158.0,26.0,...,1.000000,0.264984,89586.0,25474.0,0.413043,13.837500,83.668215,56.166667,227.0,0.117913
1,1,1,3.0,162.456962,176.506000,4.0,3530.12,20.0,18.0,27.0,...,1.009485,0.210685,3344.0,4680.0,0.520000,38.132000,100.000000,15.000000,262.0,2.261789
2,0,1,12.0,59.711176,21.246636,66.0,2273.39,107.0,85.0,18.0,...,2.968037,0.285019,21286.0,20619.0,0.450000,44.923077,52.376677,28.166667,173.0,2.981568
3,0,1,NaN,76.956000,20.618261,20.0,474.22,23.0,19.0,8.0,...,19.336538,0.190939,47842.0,52064.0,0.055556,29.780000,12.714254,3.333333,265.0,10.460627
4,0,1,2.0,49.924286,79.792000,2.0,398.96,5.0,5.0,3.0,...,1.000000,0.339576,84.0,62.0,0.305556,99.805131,100.000000,0.833333,87.0,0.550003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140027,0,0,NaN,66.557200,NaN,NaN,NaN,NaN,NaN,8.0,...,1.000000,0.200000,2853.0,2594.0,0.846154,18.070000,62.490170,0.000000,38.0,4.329561
140028,0,0,3.0,23.266667,NaN,NaN,NaN,NaN,NaN,3.0,...,9.611111,0.352941,10969.0,4763.0,0.902439,268.460000,8.020541,2.666667,233.0,3.923030
140029,1,0,NaN,11.960000,9.307568,1.0,344.38,37.0,32.0,3.0,...,3.207746,0.291667,3033.0,2490.0,0.103448,79.895000,23.080668,11.000000,375.0,1.074666
140030,0,0,1.0,23.186000,NaN,NaN,NaN,NaN,NaN,5.0,...,2.392857,0.287879,80832.0,19778.0,0.545455,17.700000,8.836698,1.333333,628.0,2.131396


In [ ]:
# @title
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Assume df is your DataFrame and 'activation' is the target variable

# Split the data into features and target
X = df.drop(columns=['activation'])
y = df['activation']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, random_state=42)

# Define a custom imputer for mean imputation excluding certain columns
class CustomImputer(SimpleImputer):
    def __init__(self, exclude_columns=[], **kwargs):
        self.exclude_columns = exclude_columns
        super().__init__(**kwargs)

    def fit(self, X, y=None):
        self.include_columns = [col for col in X.columns if col not in self.exclude_columns]
        return super().fit(X[self.include_columns], y)

    def transform(self, X):
        return super().transform(X[self.include_columns])

# Create an ensemble pipeline
ensemble_pipeline = Pipeline([
    ('imputer', ColumnTransformer([
        ('mean_imputer', CustomImputer(strategy='mean', exclude_columns=['merchant_profile_01']), X_train.columns),
        ('mode_imputer', SimpleImputer(strategy='most_frequent'), ['merchant_profile_01'])
    ])),
    ('scaler', RobustScaler()),  # Add RobustScaler for scaling numerical features
    ('features', FeatureUnion([
        ('svd', TruncatedSVD(n_components=20)),
        ('passthrough', FunctionTransformer(lambda X: X, validate=False))
    ])),
    ('ensemble', VotingClassifier(estimators=[
        ('lgbm', lgb.LGBMClassifier()),
        ('gbm', GradientBoostingClassifier()),
        ('xgb', XGBClassifier())
    ], voting='soft'))  # You can use 'hard' or 'soft' voting
])

# Fit the ensemble pipeline on the training data
ensemble_pipeline.fit(X_train, y_train)

# Predict activation on the test data
y_pred = ensemble_pipeline.predict(X_test)

# Evaluate the ensemble classifier
print(classification_report(y_test, y_pred))


[LightGBM] [Info] Number of positive: 69317, number of negative: 69314
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.651773 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 17036
[LightGBM] [Info] Number of data points in the train set: 138631, number of used features: 88
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500011 -> initscore=0.000043
[LightGBM] [Info] Start training from score 0.000043
              precision    recall  f1-score   support

           0       0.88      0.84      0.86       702
           1       0.85      0.88      0.86       699

    accuracy                           0.86      1401
   macro avg       0.86      0.86      0.86      1401
weighted avg       0.86      0.86      0.86      1401



In [ ]:
print(classification_report(y_test, y_pred,output_dict=True))

{'0': {'precision': 0.8781575037147102, 'recall': 0.8418803418803419, 'f1-score': 0.8596363636363636, 'support': 702}, '1': {'precision': 0.8475274725274725, 'recall': 0.882689556509299, 'f1-score': 0.8647512263489838, 'support': 699}, 'accuracy': 0.8622412562455389, 'macro avg': {'precision': 0.8628424881210914, 'recall': 0.8622849491948205, 'f1-score': 0.8621937949926737, 'support': 1401}, 'weighted avg': {'precision': 0.8628752825870306, 'recall': 0.8622412562455389, 'f1-score': 0.8621883186942663, 'support': 1401}}


In [ ]:
!wget 'https://d8it4huxumps7.cloudfront.net/uploads/files/amex/Amex_Campus_Challenge_Eval_Round2.csv'

--2023-09-16 13:31:24--  https://d8it4huxumps7.cloudfront.net/uploads/files/amex/Amex_Campus_Challenge_Eval_Round2.csv
Resolving d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)... 18.67.76.93, 18.67.76.108, 18.67.76.46, ...
Connecting to d8it4huxumps7.cloudfront.net (d8it4huxumps7.cloudfront.net)|18.67.76.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1307792881 (1.2G) [text/csv]
Saving to: ‘Amex_Campus_Challenge_Eval_Round2.csv.1’

Amex_Campus_Challen 100%[===================>]   1.22G   112MB/s    in 9.5s    

2023-09-16 13:31:33 (131 MB/s) - ‘Amex_Campus_Challenge_Eval_Round2.csv.1’ saved [1307792881/1307792881]



In [ ]:
reader = pd.read_csv('Amex_Campus_Challenge_Eval_Round2.csv',chunksize =10000)

In [ ]:
pipeline  = ensemble_pipeline
results = pd.DataFrame(columns=['customer', 'merchant', 'predicted_score'])
results.to_csv('submission.csv', index=False)
count =0
# Iterate through each chunk
for chunk in reader:
    print(f"Processing : {count}/{850}")
    # Store customer and merchant columns and drop them
    customer_merchant = chunk[['customer', 'merchant']]
    chunk = chunk.drop(columns=['customer', 'merchant'])

    # Predict probabilities using the pipeline for ind_recommended 0 and 1
    chunk['ind_recommended'] = 0
    # print(chunk['ind_recommended'])
    prob_0 = pipeline.predict_proba(chunk)[:, 1]

    chunk['ind_recommended'] = 1
    prob_1 = pipeline.predict_proba(chunk)[:, 1]
    # print(chunk['ind_recommended'])

    # Calculate the difference in probabilities
    predicted_score = prob_1 - prob_0
    print(prob_1,prob_0)
    # Combine results with customer and merchant information
    chunk_results = pd.DataFrame({'customer': customer_merchant['customer'],
                                  'merchant': customer_merchant['merchant'],
                                  'predicted_score': predicted_score})
    print(chunk_results)
    # Append chunk results to the main results DataFrame
    chunk_results.to_csv('submission.csv', index=False, mode='a', header=False)
    count+=1

Streaming output truncated to the last 5000 lines.
...          ...       ...              ...
989995    579783   1984828        -0.000705
989996    579783   2027219        -0.000781
989997    579783   2032133        -0.000186
989998    579783   1132656        -0.000011
989999    579783   1147197        -0.000697

[10000 rows x 3 columns]
Processing : 99/850
[0.03450797 0.06574043 0.03151525 ... 0.05671199 0.12933717 0.28824466] [0.03472989 0.07308624 0.0323869  ... 0.05860363 0.14269228 0.31501057]
        customer  merchant  predicted_score
990000    579783   1173147        -0.000222
990001    579783   1230578        -0.007346
990002    579783   1266856        -0.000872
990003    579783   1320771         0.000240
990004    579783   1547162        -0.001619
...          ...       ...              ...
999995    579840   1231686        -0.008385
999996    579840   1633860        -0.004120
999997    579840   1745081        -0.001892
999998    579840   1778626        -0.013355
999999    5

In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/Amex_Campus_Challenge_Eval_Round2.csv', chunksize=50000)
sum=0
for i in df1:
  sum += len(i['customer'])
print(sum)

4108134
